In [18]:
#Need to remove hardcoding but needed for now
%env LD_LIBRARY_PATH=/home/zach/anaconda3/envs/research/lib/
%env TF_GPU_ALLOCATOR=cuda_malloc_async

env: LD_LIBRARY_PATH=/home/zach/anaconda3/envs/research/lib/
env: TF_GPU_ALLOCATOR=cuda_malloc_async


In [19]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
import re
from tqdm import tqdm
from tensorflow import keras
import os
from official.nlp import optimization  # to create AdamW optimizer
import tensorflow_addons as tfa
f1 = tfa.metrics.F1Score(num_classes=1,average=None)

tf.get_logger().setLevel('ERROR')



In [20]:
print(tf.__version__)

2.9.0


In [21]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [22]:
os.environ['LD_LIBRARY_PATH']

'/home/zach/anaconda3/envs/research/lib/'

## Data Import and Cleaning

In [23]:
train = pd.read_csv('data/hateval2019_en_train.csv')
test = pd.read_csv('data/hateval2019_en_test.csv')
val = pd.read_csv('data/hateval2019_en_dev.csv')

train = train.drop(['TR','AG'],1)
test = test.drop(['TR','AG'],1)
val = val.drop(['TR','AG'],1)

/tmp/ipykernel_64834/675437588.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  train = train.drop(['TR','AG'],1)
/tmp/ipykernel_64834/675437588.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  test = test.drop(['TR','AG'],1)
/tmp/ipykernel_64834/675437588.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  val = val.drop(['TR','AG'],1)


In [24]:
train.head()


,id,text,HS
0,201,"Hurray, saving us $$$ in so many ways @potus @...",1
1,202,Why would young fighting age men be the vast m...,1
2,203,@KamalaHarris Illegals Dump their Kids at the ...,1
3,204,NY Times: 'Nearly All White' States Pose 'an A...,0
4,205,Orban in Brussels: European leaders are ignori...,0


In [25]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Embedding, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.regularizers import L1,L2, l1_l2
import io

In [26]:
def normalize_tweet(text):
    """
    Removes hashtags, @s, links, and punctuation
    :param text:Text to be cleaned
    :return: text with mentions, hashtages, and urls removes
    """
    processed_text = text.lower()
    processed_text = re.sub(r"(?:\@|http?\://|https?\://|www|t\.)\S+", "", processed_text)
    processed_text = re.sub(r"(?:\.|,|\?|-)", " ", processed_text)
    processed_text = re.sub(r"(?:\@|http?\://|https?\://|www|\.com)", "", processed_text)
    processed_text = re.sub(r'[^\w\s]', '', processed_text)
    processed_text = " ".join(processed_text.split())
    return processed_text

def x_y_split(data):
    """splits and X and y from dataframe

    Args:
        data:dataframe to split from

    Returns:
        tuple:X data, y data
    """
    X = data['text']
    X = X.apply(normalize_tweet)
    y = data['HS']
    return X, y

## Split sequences into train, validation, and test sets

In [27]:
#Split x and ys
x_train, y_train = x_y_split(train)
print(x_train)
#x_train.to_csv('data/x_train.csv')
#y_train.to_csv('data/y_train.csv')

0       hurray saving us in so many ways lockthemup bu...
1       why would young fighting age men be the vast m...
2       illegals dump their kids at the border like ro...
3       ny times nearly all white states pose an array...
4       orban in brussels european leaders are ignorin...
                              ...                        
8995                  i am proud to be a hysterical woman
8996    hollywood is complicit in the rape and sexual ...
8997    what a fucking cunt i hate seeing kids getting...
8998                                hysterical woman like
8999    nearly every woman i know has metoo in their f...
Name: text, Length: 9000, dtype: object


In [28]:
#Split x and ys
x_test, y_test = x_y_split(test)
print(x_test)
#x_test.to_csv('data/x_test.csv')
#y_test.to_csv('data/y_test.csv')

0       oh i could have gone on about taxes since the ...
1       several of the wild fires in california and co...
2       my question is how do you resettle a refugee a...
3       europe youve got a problem we must hurry and b...
4       this is outrageous stopillegalimmigration meri...
                              ...                        
2995    you can never take a l off a real bitch im hot...
2996    likes to call me a cunt a bitch but i tell him...
2997    1 never said you were taught 2 you called me b...
2998    if i see and know you a hoe why would i hit yo...
2999     you be chasing them hoes fuck what a bitch think
Name: text, Length: 3000, dtype: object


In [29]:
#Split x and ys
x_val, y_val = x_y_split(val)
print(x_val)
#x_val.to_csv('data/x_val.csv')
#y_val.to_csv('data/y_val.csv')

0      i swear im getting to places just in the nick ...
1      im an immigrant and trump is right on immigrat...
2      illegalimmigrants illegalaliens electoralsyste...
3      we have our own invasion issues with mexicans ...
4      worker charged with sexually molesting eight c...
                             ...                        
995                     you unfollowed me fuck you pussy
996    stfu bitch and you go make some satanic music ...
997    honey as a fellow white chick let me tell you ...
998    i hate bitches who talk about niggaz with kids...
999    you won the life time recipient for hysterical...
Name: text, Length: 1000, dtype: object


In [30]:
import tensorflow_hub as hub
import tensorflow_text as text  # A dependency of the preprocessing model

In [31]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [32]:
tfhub_handle_encoder = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

In [33]:
def build_classifier_model(do,trainable=False):
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=trainable, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    net = tf.keras.layers.Dropout(do)(net)
    net = tf.keras.layers.Dense(1, activation='sigmoid', name='classifier')(net)
    return tf.keras.Model(text_input, net)




In [34]:
loss = tf.keras.losses.BinaryCrossentropy()
learning_rates = [5e-5, 1e-4, 3e-5, 3e-5]
learning_rates_text = ['5e-5', '1e-4', '3e-5', '3e-5']
batch_sizes = [8, 16]
dropout = [.5,.75,.25,0.0,.1]
results = {}
results_acc = {}
best_epochs = {}
for do in dropout:
    for bs in batch_sizes:
        for lr in range(len(learning_rates)):
            np.random.seed(42)
            tf.random.set_seed(42)
            string = 'lr{lr}_bs{bs}_do{do}'.format(lr=learning_rates_text[lr], bs=bs, do=do)

            bert = build_classifier_model(do, trainable=True)
            bert.compile(loss=loss, optimizer=Adam(learning_rate=learning_rates[lr]), metrics=[f1, 'accuracy'])
            history = bert.fit(x=x_train,
                               y=y_train, batch_size=bs,
                               validation_data=(x_val, y_val),
                               epochs=4)

            max_f1 = max(history.history['val_f1_score'])
            max_acc = max(history.history['val_accuracy'])
            val_acc_per_epoch = history.history['val_accuracy']
            best_epoch = val_acc_per_epoch.index(max_acc) + 1
            results[string] = max_f1
            results_acc[string] = max_acc
            best_epochs[string] = best_epoch


Epoch 1/4
1125/1125 [==============================] - 184s 158ms/step - loss: 0.5035 - f1_score: 0.5919 - accuracy: 0.7560 - val_loss: 0.4812 - val_f1_score: 0.5985 - val_accuracy: 0.7770
Epoch 2/4
1125/1125 [==============================] - 182s 162ms/step - loss: 0.3713 - f1_score: 0.5919 - accuracy: 0.8406 - val_loss: 0.5690 - val_f1_score: 0.5985 - val_accuracy: 0.7760
Epoch 3/4
1125/1125 [==============================] - 190s 169ms/step - loss: 0.2101 - f1_score: 0.5919 - accuracy: 0.9211 - val_loss: 0.6628 - val_f1_score: 0.5985 - val_accuracy: 0.7710
Epoch 4/4
1125/1125 [==============================] - 189s 168ms/step - loss: 0.1205 - f1_score: 0.5919 - accuracy: 0.9571 - val_loss: 0.8040 - val_f1_score: 0.5985 - val_accuracy: 0.7410
Epoch 1/4
1125/1125 [==============================] - 193s 166ms/step - loss: 0.6995 - f1_score: 0.5925 - accuracy: 0.6261 - val_loss: 0.7360 - val_f1_score: 0.5985 - val_accuracy: 0.5730
Epoch 2/4
1125/1125 [==============================] - 

In [35]:
results

{'lr5e-5_bs8_do0.5': array([0.5984583], dtype=float32),
 'lr1e-4_bs8_do0.5': array([0.5984583], dtype=float32),
 'lr3e-5_bs8_do0.5': array([0.5984583], dtype=float32),
 'lr5e-5_bs16_do0.5': array([0.5984583], dtype=float32),
 'lr1e-4_bs16_do0.5': array([0.5984583], dtype=float32),
 'lr3e-5_bs16_do0.5': array([0.5984583], dtype=float32),
 'lr5e-5_bs8_do0.75': array([0.5984583], dtype=float32),
 'lr1e-4_bs8_do0.75': array([0.5984583], dtype=float32),
 'lr3e-5_bs8_do0.75': array([0.5984583], dtype=float32),
 'lr5e-5_bs16_do0.75': array([0.5984583], dtype=float32),
 'lr1e-4_bs16_do0.75': array([0.5984583], dtype=float32),
 'lr3e-5_bs16_do0.75': array([0.5984583], dtype=float32),
 'lr5e-5_bs8_do0.25': array([0.5984583], dtype=float32),
 'lr1e-4_bs8_do0.25': array([0.5984583], dtype=float32),
 'lr3e-5_bs8_do0.25': array([0.5984583], dtype=float32),
 'lr5e-5_bs16_do0.25': array([0.5984583], dtype=float32),
 'lr1e-4_bs16_do0.25': array([0.5984583], dtype=float32),
 'lr3e-5_bs16_do0.25': array(

In [36]:
results2 = {}
results2_acc = {}
best_epochs2 = {}

In [37]:
for do in dropout:
    for bs in batch_sizes:
        for lr in range(len(learning_rates)):
            np.random.seed(42)
            tf.random.set_seed(42)
            string = 'lr{lr}_bs{bs}_do{do}'.format(lr=learning_rates_text[lr],bs=bs,do=do)
            #filepath = 'checkpoints/bert_finetuned/lr{lr}_bs{bs}.hdf5'.format(lr=learning_rates_text[lr],bs=bs)
            #checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=filepath,
                                                 #    monitor='val_f1_score',
                                                  #  save_best_only=True,
                                                   #  verbose=1,)
            #callbacks = [checkpoint]

            bert = build_classifier_model(do,trainable=False)
            bert.compile(loss = loss, optimizer = Adam(learning_rate=learning_rates[lr]), metrics = [f1,'accuracy'])
            history = bert.fit(x=x_train,
                                y=y_train,batch_size=bs,
                                validation_data=(x_val,y_val),
                                epochs=4)

            max_f1 = max(history.history['val_f1_score'])
            max_acc = max(history.history['val_accuracy'])
            results2[string] = max_f1
            results2_acc[string] = max_acc
            val_acc_per_epoch = history.history['val_accuracy']
            best_epoch = val_acc_per_epoch.index(max_acc) + 1
            best_epochs2[string] = best_epoch

Epoch 1/4
1125/1125 [==============================] - 63s 53ms/step - loss: 0.7632 - f1_score: 0.5925 - accuracy: 0.5301 - val_loss: 0.6809 - val_f1_score: 0.5985 - val_accuracy: 0.5660
Epoch 2/4
1125/1125 [==============================] - 60s 53ms/step - loss: 0.7418 - f1_score: 0.5919 - accuracy: 0.5403 - val_loss: 0.6722 - val_f1_score: 0.5985 - val_accuracy: 0.5730
Epoch 3/4
1125/1125 [==============================] - 60s 53ms/step - loss: 0.7353 - f1_score: 0.5919 - accuracy: 0.5449 - val_loss: 0.6652 - val_f1_score: 0.5985 - val_accuracy: 0.5820
Epoch 4/4
1125/1125 [==============================] - 60s 53ms/step - loss: 0.7267 - f1_score: 0.5919 - accuracy: 0.5452 - val_loss: 0.6605 - val_f1_score: 0.5985 - val_accuracy: 0.5870
Epoch 1/4
1125/1125 [==============================] - 64s 53ms/step - loss: 0.7556 - f1_score: 0.5925 - accuracy: 0.5333 - val_loss: 0.6717 - val_f1_score: 0.5985 - val_accuracy: 0.5740
Epoch 2/4
1125/1125 [==============================] - 60s 54ms/s

In [38]:
results2

{'lr5e-5_bs8_do0.5': array([0.5984583], dtype=float32),
 'lr1e-4_bs8_do0.5': array([0.5984583], dtype=float32),
 'lr3e-5_bs8_do0.5': array([0.5984583], dtype=float32),
 'lr5e-5_bs16_do0.5': array([0.5984583], dtype=float32),
 'lr1e-4_bs16_do0.5': array([0.5984583], dtype=float32),
 'lr3e-5_bs16_do0.5': array([0.5984583], dtype=float32),
 'lr5e-5_bs8_do0.75': array([0.5984583], dtype=float32),
 'lr1e-4_bs8_do0.75': array([0.5984583], dtype=float32),
 'lr3e-5_bs8_do0.75': array([0.5984583], dtype=float32),
 'lr5e-5_bs16_do0.75': array([0.5984583], dtype=float32),
 'lr1e-4_bs16_do0.75': array([0.5984583], dtype=float32),
 'lr3e-5_bs16_do0.75': array([0.5984583], dtype=float32),
 'lr5e-5_bs8_do0.25': array([0.5984583], dtype=float32),
 'lr1e-4_bs8_do0.25': array([0.5984583], dtype=float32),
 'lr3e-5_bs8_do0.25': array([0.5984583], dtype=float32),
 'lr5e-5_bs16_do0.25': array([0.5984583], dtype=float32),
 'lr1e-4_bs16_do0.25': array([0.5984583], dtype=float32),
 'lr3e-5_bs16_do0.25': array(

In [39]:
results_acc


{'lr5e-5_bs8_do0.5': 0.7770000100135803,
 'lr1e-4_bs8_do0.5': 0.5730000138282776,
 'lr3e-5_bs8_do0.5': 0.781000018119812,
 'lr5e-5_bs16_do0.5': 0.7630000114440918,
 'lr1e-4_bs16_do0.5': 0.7620000243186951,
 'lr3e-5_bs16_do0.5': 0.7639999985694885,
 'lr5e-5_bs8_do0.75': 0.7889999747276306,
 'lr1e-4_bs8_do0.75': 0.5730000138282776,
 'lr3e-5_bs8_do0.75': 0.7749999761581421,
 'lr5e-5_bs16_do0.75': 0.7559999823570251,
 'lr1e-4_bs16_do0.75': 0.5730000138282776,
 'lr3e-5_bs16_do0.75': 0.7699999809265137,
 'lr5e-5_bs8_do0.25': 0.777999997138977,
 'lr1e-4_bs8_do0.25': 0.5730000138282776,
 'lr3e-5_bs8_do0.25': 0.7879999876022339,
 'lr5e-5_bs16_do0.25': 0.7829999923706055,
 'lr1e-4_bs16_do0.25': 0.7549999952316284,
 'lr3e-5_bs16_do0.25': 0.7870000004768372,
 'lr5e-5_bs8_do0.0': 0.7730000019073486,
 'lr1e-4_bs8_do0.0': 0.609000027179718,
 'lr3e-5_bs8_do0.0': 0.7889999747276306,
 'lr5e-5_bs16_do0.0': 0.7699999809265137,
 'lr1e-4_bs16_do0.0': 0.7580000162124634,
 'lr3e-5_bs16_do0.0': 0.7810000181198

In [40]:
results2_acc

{'lr5e-5_bs8_do0.5': 0.5870000123977661,
 'lr1e-4_bs8_do0.5': 0.6019999980926514,
 'lr3e-5_bs8_do0.5': 0.578000009059906,
 'lr5e-5_bs16_do0.5': 0.5799999833106995,
 'lr1e-4_bs16_do0.5': 0.5889999866485596,
 'lr3e-5_bs16_do0.5': 0.5720000267028809,
 'lr5e-5_bs8_do0.75': 0.5809999704360962,
 'lr1e-4_bs8_do0.75': 0.5899999737739563,
 'lr3e-5_bs8_do0.75': 0.5720000267028809,
 'lr5e-5_bs16_do0.75': 0.5770000219345093,
 'lr1e-4_bs16_do0.75': 0.5870000123977661,
 'lr3e-5_bs16_do0.75': 0.5699999928474426,
 'lr5e-5_bs8_do0.25': 0.5879999995231628,
 'lr1e-4_bs8_do0.25': 0.6119999885559082,
 'lr3e-5_bs8_do0.25': 0.578000009059906,
 'lr5e-5_bs16_do0.25': 0.5820000171661377,
 'lr1e-4_bs16_do0.25': 0.5889999866485596,
 'lr3e-5_bs16_do0.25': 0.5730000138282776,
 'lr5e-5_bs8_do0.0': 0.5929999947547913,
 'lr1e-4_bs8_do0.0': 0.6240000128746033,
 'lr3e-5_bs8_do0.0': 0.5860000252723694,
 'lr5e-5_bs16_do0.0': 0.5879999995231628,
 'lr1e-4_bs16_do0.0': 0.6010000109672546,
 'lr3e-5_bs16_do0.0': 0.574999988079

In [42]:
max(results_acc)

'lr5e-5_bs8_do0.75'

In [45]:
results_acc['lr5e-5_bs8_do0.75']
#best_epochs['lr5e-5_bs8_do0.75']

0.7889999747276306

In [46]:
from sklearn.metrics import classification_report
#model = tf.keras.models.load_model('checkpoints/fast_text_model.hdf5')
#print(val)
tf.random.set_seed(42)
np.random.seed(42)
bert = build_classifier_model(.75,True)
bert.compile(loss = loss, optimizer = Adam(learning_rate=5e-5), metrics = [f1,'accuracy'])
history = bert.fit(x=x_train,
                        y=y_train,batch_size=8,
                        validation_data=(x_val,y_val),
                        epochs=2)


Epoch 1/2
1125/1125 [==============================] - 183s 157ms/step - loss: 0.5403 - f1_score: 0.5910 - accuracy: 0.7311 - val_loss: 0.5143 - val_f1_score: 0.5985 - val_accuracy: 0.7460
Epoch 2/2
1125/1125 [==============================] - 180s 160ms/step - loss: 0.3698 - f1_score: 0.5919 - accuracy: 0.8391 - val_loss: 0.4940 - val_f1_score: 0.5985 - val_accuracy: 0.7790


In [49]:
pred = bert.predict(x_test)
pred = np.round(pred)
#print(pred)
print(classification_report(y_test,pred))

94/94 [==============================] - 18s 195ms/step
              precision    recall  f1-score   support

           0       0.88      0.22      0.35      1740
           1       0.47      0.96      0.63      1260

    accuracy                           0.53      3000
   macro avg       0.67      0.59      0.49      3000
weighted avg       0.71      0.53      0.47      3000



In [51]:
max(results2_acc)

'lr5e-5_bs8_do0.75'

In [53]:
results2_acc['lr5e-5_bs8_do0.75']
best_epochs2['lr5e-5_bs8_do0.75']

4

In [55]:
tf.random.set_seed(42)
np.random.seed(42)
bert = build_classifier_model(.75,False)
bert.compile(loss = loss, optimizer = Adam(learning_rate=5e-5), metrics = [f1,'accuracy'])
history = bert.fit(x=x_train,
                        y=y_train,batch_size=8,
                        validation_data=(x_val,y_val),
                        epochs=4)

Epoch 1/4
1125/1125 [==============================] - 70s 59ms/step - loss: 0.8916 - f1_score: 0.5920 - accuracy: 0.5192 - val_loss: 0.6873 - val_f1_score: 0.5985 - val_accuracy: 0.5710
Epoch 2/4
1125/1125 [==============================] - 63s 56ms/step - loss: 0.8536 - f1_score: 0.5919 - accuracy: 0.5252 - val_loss: 0.6787 - val_f1_score: 0.5985 - val_accuracy: 0.5680
Epoch 3/4
1125/1125 [==============================] - 63s 56ms/step - loss: 0.8271 - f1_score: 0.5919 - accuracy: 0.5349 - val_loss: 0.6728 - val_f1_score: 0.5985 - val_accuracy: 0.5760
Epoch 4/4
1125/1125 [==============================] - 63s 56ms/step - loss: 0.8229 - f1_score: 0.5919 - accuracy: 0.5274 - val_loss: 0.6676 - val_f1_score: 0.5985 - val_accuracy: 0.5810


In [56]:
pred = bert.predict(x_test)
pred = np.round(pred)
print(classification_report(y_test,pred))

94/94 [==============================] - 18s 189ms/step
              precision    recall  f1-score   support

           0       0.59      0.94      0.73      1740
           1       0.55      0.10      0.17      1260

    accuracy                           0.59      3000
   macro avg       0.57      0.52      0.45      3000
weighted avg       0.57      0.59      0.49      3000



In [58]:
pred

array([[0.],
       [1.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)

In [60]:
y_test

0       0
1       0
2       0
3       1
4       1
       ..
2995    1
2996    1
2997    1
2998    1
2999    1
Name: HS, Length: 3000, dtype: int64